<a href="https://colab.research.google.com/github/JoseAlbertoSeco/BlablacarCarpooling_DataMining/blob/Patterns/notebooks/Clasificaci%C3%B3n_multiclase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación multiclase

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
from sklearn import linear_model
from sklearn import tree 

from sklearn.metrics import plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.multiclass import unique_labels

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
clm = pd.read_csv("/content/drive/MyDrive/Datos/df_castilla.csv") 
clm.head(2)

,Unnamed: 0,DIA,AÑO,SEMANA,MES,PAIS,ORIGEN,DESTINO,ASIENTOS_OFERTADOS,ASIENTOS_CONFIRMADOS,VIAJES_OFERTADOS,VIAJES_CONFIRMADOS,OFERTANTES,OFERTANTES_NUEVOS,CA_ORIGEN,CA_DESTINO,PROVINCIA_ORIGEN,PROVINCIA_DESTINO,INTERPROVINCIAL,Festivo
0,453525,2017-11-05,2017,44,11,es,Albacete,Guadalajara,11.0,1,6,1,6,1,Castilla-La Mancha,Castilla-La Mancha,Albacete,Guadalajara,No,No
1,453526,2017-11-12,2017,45,11,es,Albacete,Guadalajara,11.0,1,6,1,6,1,Castilla-La Mancha,Castilla-La Mancha,Albacete,Guadalajara,No,No


In [ ]:
andalucia = pd.read_csv("/content/drive/MyDrive/Datos/df_andalucia.csv") 
andalucia.head(2)

,Unnamed: 0,DIA,AÑO,SEMANA,MES,PAIS,ORIGEN,DESTINO,ASIENTOS_OFERTADOS,ASIENTOS_CONFIRMADOS,VIAJES_OFERTADOS,VIAJES_CONFIRMADOS,OFERTANTES,OFERTANTES_NUEVOS,CA_ORIGEN,CA_DESTINO,PROVINCIA_ORIGEN,PROVINCIA_DESTINO,INTERPROVINCIAL,Festivo
0,285649,2017-11-01,2017,44,11,es,Albox,Granada,1.0,1,1,1,1,0,Andalucía,Andalucía,Almería,Granada,No,Si
1,285650,2017-11-03,2017,44,11,es,Albox,Granada,12.0,1,4,1,4,0,Andalucía,Andalucía,Almería,Granada,No,No


In [ ]:
clm = clm.drop(clm.columns[[0]], axis=1)
andalucia = andalucia.drop(andalucia.columns[[0]], axis=1)

# Label Encoding

## Andalucía

In [ ]:
andalucia_encoded = andalucia.copy()

In [ ]:
andalucia_encoded['ORIGEN_E'] = andalucia['ORIGEN'].astype('category').cat.codes
andalucia_encoded['DESTINO_E'] = andalucia['DESTINO'].astype('category').cat.codes
andalucia_encoded['PROVINCIA_ORIGEN_E'] = andalucia['PROVINCIA_ORIGEN'].astype('category').cat.codes
andalucia_encoded['PROVINCIA_DESTINO_E'] = andalucia['PROVINCIA_DESTINO'].astype('category').cat.codes
andalucia_encoded['INTERPROVINCIAL_E'] = andalucia['INTERPROVINCIAL'].astype('category').cat.codes
andalucia_encoded['FESTIVO'] = andalucia_encoded['Festivo'].astype('category').cat.codes

In [ ]:
andalucia_encoded.tail(2)

,DIA,AÑO,SEMANA,MES,PAIS,ORIGEN,DESTINO,ASIENTOS_OFERTADOS,ASIENTOS_CONFIRMADOS,VIAJES_OFERTADOS,VIAJES_CONFIRMADOS,OFERTANTES,OFERTANTES_NUEVOS,CA_ORIGEN,CA_DESTINO,PROVINCIA_ORIGEN,PROVINCIA_DESTINO,INTERPROVINCIAL,Festivo,ORIGEN_E,DESTINO_E,PROVINCIA_ORIGEN_E,PROVINCIA_DESTINO_E,INTERPROVINCIAL_E,FESTIVO
246053,2018-06-08,2018,23,6,es,Mairena del Aljarafe,Corteconcepción,3.0,2,1,1,1,1,Andalucía,Andalucía,Sevilla,Huelva,No,No,296,142,8,4,0,0
246054,2018-07-06,2018,27,7,es,Tomares,Algámitas,4.0,1,1,1,1,0,Andalucía,Andalucía,Sevilla,Sevilla,Si,No,428,25,8,8,1,0


In [ ]:
andalucia_encoded = andalucia_encoded.drop(labels=['ORIGEN','Festivo', 'DESTINO', 'CA_ORIGEN', 'CA_DESTINO', 'PROVINCIA_ORIGEN', 'PROVINCIA_DESTINO', 'INTERPROVINCIAL', 'PAIS'], axis=1)

In [ ]:
andalucia_encoded = andalucia_encoded.rename(columns={'ORIGEN_E':'ORIGEN','DESTINO_E':'DESTINO', 'CA_ORIGEN_E':'CA_ORIGEN', 'CA_DESTINO_E': 'CA_DESTINO','PROVINCIA_ORIGEN_E': 'PROVINCIA_ORIGEN','PROVINCIA_DESTINO_E': 'PROVINCIA_DESTINO','INTERPROVINCIAL_E': 'INTERPROVINCIAL'})

In [ ]:
andalucia_encoded.head(2)

,DIA,AÑO,SEMANA,MES,ASIENTOS_OFERTADOS,ASIENTOS_CONFIRMADOS,VIAJES_OFERTADOS,VIAJES_CONFIRMADOS,OFERTANTES,OFERTANTES_NUEVOS,ORIGEN,DESTINO,PROVINCIA_ORIGEN,PROVINCIA_DESTINO,INTERPROVINCIAL,FESTIVO
0,2017-11-01,2017,44,11,1.0,1,1,1,1,0,6,218,0,3,0,1
1,2017-11-03,2017,44,11,12.0,1,4,1,4,0,6,218,0,3,0,0


## Castilla-la Mancha

In [ ]:
clm_encoded = clm.copy()

In [ ]:
clm_encoded['ORIGEN_E'] = clm['ORIGEN'].astype('category').cat.codes
clm_encoded['DESTINO_E'] = clm['DESTINO'].astype('category').cat.codes
clm_encoded['PROVINCIA_ORIGEN_E'] = clm['PROVINCIA_ORIGEN'].astype('category').cat.codes
clm_encoded['PROVINCIA_DESTINO_E'] = clm['PROVINCIA_DESTINO'].astype('category').cat.codes
clm_encoded['INTERPROVINCIAL_E'] = clm['INTERPROVINCIAL'].astype('category').cat.codes
clm_encoded['FESTIVO'] = clm_encoded['Festivo'].astype('category').cat.codes

In [ ]:
clm_encoded.tail(2)

,DIA,AÑO,SEMANA,MES,PAIS,ORIGEN,DESTINO,ASIENTOS_OFERTADOS,ASIENTOS_CONFIRMADOS,VIAJES_OFERTADOS,VIAJES_CONFIRMADOS,OFERTANTES,OFERTANTES_NUEVOS,CA_ORIGEN,CA_DESTINO,PROVINCIA_ORIGEN,PROVINCIA_DESTINO,INTERPROVINCIAL,Festivo,ORIGEN_E,DESTINO_E,PROVINCIA_ORIGEN_E,PROVINCIA_DESTINO_E,INTERPROVINCIAL_E,FESTIVO
20860,2018-07-14,2018,28,7,es,Guadalajara,Valdelcubo,3.0,1,1,1,1,0,Castilla-La Mancha,Castilla-La Mancha,Guadalajara,Guadalajara,Si,No,52,139,4,4,1,0
20861,2018-08-10,2018,32,8,es,Guadalajara,Cobeta,5.0,1,2,1,1,0,Castilla-La Mancha,Castilla-La Mancha,Guadalajara,Guadalajara,Si,No,52,44,4,4,1,0


In [ ]:
clm_encoded = clm_encoded.drop(labels=['ORIGEN','Festivo', 'DESTINO', 'CA_ORIGEN', 'CA_DESTINO', 'PROVINCIA_ORIGEN', 'PROVINCIA_DESTINO', 'INTERPROVINCIAL', 'PAIS'], axis=1)

In [ ]:
clm_encoded = clm_encoded.rename(columns={'ORIGEN_E':'ORIGEN','DESTINO_E':'DESTINO', 'CA_ORIGEN_E':'CA_ORIGEN', 'CA_DESTINO_E': 'CA_DESTINO','PROVINCIA_ORIGEN_E': 'PROVINCIA_ORIGEN','PROVINCIA_DESTINO_E': 'PROVINCIA_DESTINO','INTERPROVINCIAL_E': 'INTERPROVINCIAL'})

In [ ]:
clm_encoded.head(2)

,DIA,AÑO,SEMANA,MES,ASIENTOS_OFERTADOS,ASIENTOS_CONFIRMADOS,VIAJES_OFERTADOS,VIAJES_CONFIRMADOS,OFERTANTES,OFERTANTES_NUEVOS,ORIGEN,DESTINO,PROVINCIA_ORIGEN,PROVINCIA_DESTINO,INTERPROVINCIAL,FESTIVO
0,2017-11-05,2017,44,11,11.0,1,6,1,6,1,1,62,0,4,0,0
1,2017-11-12,2017,45,11,11.0,1,6,1,6,1,1,62,0,4,0,0


# Groupby destino a modo de labels

## Andalucia

In [ ]:
df_andalucia = andalucia_encoded.copy()

In [ ]:
df_and = df_andalucia.groupby(['DIA', 'ORIGEN','VIAJES_OFERTADOS','ASIENTOS_OFERTADOS','MES','AÑO','SEMANA','INTERPROVINCIAL']).DESTINO.sum()
df_and = pd.DataFrame(df_and)
df_and

DESTINO
DIA        ORIGEN VIAJES_OFERTADOS ASIENTOS_OFERTADOS MES AÑO  SEMANA INTERPROVINCIAL         
2017-11-01 6      1                1.0                11  2017 44     0                    218
           13     1                3.0                11  2017 44     1                    157
           18     1                1.0                11  2017 44     0                    237
                                   3.0                11  2017 44     0                     29
                  3                6.0                11  2017 44     0                    157
...                                                                                        ...
2019-10-31 484    2                6.0                10  2019 44     0                    347
                  3                4.0                10  2019 44     0                     35
                  7                17.0               10  2019 44     0                    157
                  10               24.0               10  2019 44     0                    437
                  15               32.0               10  2019 44     0                    218

[228261 rows x 1 columns]

In [ ]:
df_and = df_and.reset_index()

In [ ]:
df_and = df_and.set_index("DIA")

In [ ]:
df_and[df_and < 0] = 0

df_and

,ORIGEN,VIAJES_OFERTADOS,ASIENTOS_OFERTADOS,MES,AÑO,SEMANA,INTERPROVINCIAL,DESTINO
DIA,,,,,,,,
2017-11-01,6,1,1.0,11,2017,44,0,218
2017-11-01,13,1,3.0,11,2017,44,1,157
2017-11-01,18,1,1.0,11,2017,44,0,237
2017-11-01,18,1,3.0,11,2017,44,0,29
2017-11-01,18,3,6.0,11,2017,44,0,157
...,...,...,...,...,...,...,...,...
2019-10-31,484,2,6.0,10,2019,44,0,347
2019-10-31,484,3,4.0,10,2019,44,0,35
2019-10-31,484,7,17.0,10,2019,44,0,157


## Castilla-la Mancha

In [ ]:
df_clm = clm_encoded.copy()

In [ ]:
df_clm = df_clm.groupby(['DIA', 'ORIGEN','VIAJES_OFERTADOS','ASIENTOS_OFERTADOS','MES','AÑO','SEMANA','INTERPROVINCIAL']).DESTINO.sum()
df_clm = pd.DataFrame(df_clm)
df_clm

DESTINO
DIA        ORIGEN VIAJES_OFERTADOS ASIENTOS_OFERTADOS MES AÑO  SEMANA INTERPROVINCIAL         
2017-11-01 1      2                6.0                11  2017 44     0                     78
                  5                13.0               11  2017 44     1                     10
                  8                19.0               11  2017 44     0                    269
                  11               30.0               11  2017 44     1                    153
           6      3                7.0                11  2017 44     1                     42
...                                                                                        ...
2019-10-31 118    20               50.0               10  2019 44     1                     42
                  21               42.0               10  2019 44     0                      1
           122    8                16.0               10  2019 44     0                    134
           133    13               27.0               10  2019 44     0                     42
                  27               60.0               10  2019 44     1                      1

[20496 rows x 1 columns]

In [ ]:
df_clm = df_clm.reset_index()

In [ ]:
df_clm = df_clm.set_index("DIA")

# Naive Bayes

In [ ]:
X = df_and.drop(labels=['DESTINO'], axis=1)
y = df_and[['DESTINO']]

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.32, random_state=42)

In [ ]:
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, ComplementNB

# classifiers
names = ["GaussianNB", "MultiNomialNB", 'BernouilliNB', 'ComplementNB']
classifiers = [GaussianNB(), MultinomialNB(), BernoulliNB(), ComplementNB()]

for name, clf in zip(names, classifiers):
  # FIT THE MODEL
  clf.fit(X_train, y_train.values.ravel())
  # PREDIT AND SCORE  
  score = clf.score(X_test, y_test) #Returns the mean accuracy on the given test data and labels.
  print ("Modelo: %s = %6.2f" % (name, score))

Modelo: GaussianNB =   0.04
Modelo: MultiNomialNB =   0.02
Modelo: BernouilliNB =   0.13
Modelo: ComplementNB =   0.07


In [ ]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(X_train, y_train.values.ravel())
y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=clf.classes_)
print(cm)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
from sklearn import metrics
print ("MAE:", metrics.mean_absolute_error(y_pred , y_test))
print ("MSE:", metrics.mean_squared_error(y_pred , y_test))

MAE: 165.3363999780954
MSE: 2713.182520124856


In [ ]:
from statistics import mode

In [ ]:
res = clf.predict(X_test)


437


# RandomForest Classifier

In [ ]:
X = df_clm.drop(labels=['DESTINO'], axis=1)
y = df_clm[['DESTINO']]

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.32, random_state=42)

In [ ]:
import numpy as np
def report(results, n_top=3): # Función para mostrar resultados
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint as sp_randint


clf_clm = RandomForestClassifier(n_jobs=-1, random_state=42)

param_dist = { "n_estimators" : [2, 4, 8, 16, 32, 64, 128],
              "max_features": ['auto', 'sqrt'], 
              "max_depth": [16, 8, 4, 2], 
              "min_samples_split": sp_randint(2, 40), 
              "min_samples_leaf": sp_randint(1, 40), 
              "bootstrap": [True, False], 
            }


random_search = RandomizedSearchCV(clf_clm, scoring= 'f1_micro', 
                                   param_distributions=param_dist, cv = 5, 
                                   n_iter= 40)
random_search.fit(X_train, y_train.values.ravel())

report(random_search.cv_results_)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


Model with rank: 1
Mean validation score: 0.446 (std: 0.011)
Parameters: {'bootstrap': False, 'max_depth': 16, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 26, 'n_estimators': 32}

Model with rank: 2
Mean validation score: 0.439 (std: 0.011)
Parameters: {'bootstrap': True, 'max_depth': 16, 'max_features': 'sqrt', 'min_samples_leaf': 9, 'min_samples_split': 27, 'n_estimators': 16}

Model with rank: 3
Mean validation score: 0.439 (std: 0.012)
Parameters: {'bootstrap': False, 'max_depth': 16, 'max_features': 'sqrt', 'min_samples_leaf': 6, 'min_samples_split': 19, 'n_estimators': 64}



In [ ]:
clf_rf = random_search.best_estimator_
clf_rf.fit(X_train, y_train.values.ravel()) # Construcción del modelo

preds_rf = clf_rf.predict(X_test) # Test del modelo

In [ ]:
from sklearn import metrics
print ("MAE:", metrics.mean_absolute_error(preds_rf , y_test))
print ("MSE:", metrics.mean_squared_error(preds_rf , y_test))

MAE: 40.65284342125324
MSE: 4157.78014941302


In [ ]:
# FEATURE IMPORTANCES
print ('Feature Importances')
pd.DataFrame({'Attributes': ('ORIGEN','VIAJES_OFERTADOS','ASIENTOS_OFERTADOS','MES','AÑO','SEMANA','INTERPROVINCIAL') ,
              'Decision Tree': clf_rf.feature_importances_})

Feature Importances


,Attributes,Decision Tree
0,ORIGEN,0.509902
1,VIAJES_OFERTADOS,0.115210
2,ASIENTOS_OFERTADOS,0.129125
3,MES,0.041756
4,AÑO,0.019740
5,SEMANA,0.072901
6,INTERPROVINCIAL,0.111366


## Predicción para el usuario en Castill-la Mancha

In [ ]:
from collections import Counter

In [ ]:
orig_clm = clm.ORIGEN.astype('category')
origen_dic_clm = dict(enumerate(orig_clm.cat.categories))

desti_clm = clm.DESTINO.astype('category')
destino_dic_clm = dict(enumerate(desti_clm.cat.categories))

interprovincial_clm = clm.INTERPROVINCIAL.astype('category')
interprovincial_dic_clm =dict(enumerate(interprovincial_clm.cat.categories))

In [ ]:
#@title ORIGEN

origen_clm = 'Ciudad Real' #@param {type:"string"}

origen_clm = list(origen_dic_clm.keys())[list(origen_dic_clm.values()).index(origen_clm)]
origen_clm

37

In [ ]:
from datetime import datetime

#@title FECHA DE VIAJE
dia_clm = '2022-01-25' #@param {type:"date"}


date_clm = pd.Timestamp(dia_clm)
mes_clm = date_clm.month

semana_clm = date_clm.week
año_clm = date_clm.year
print(mes_clm,año_clm,semana_clm)

1 2022 4


In [ ]:
#@title Interprovincial

interprovincial_clm = False #@param ["False", "True"] {type:"raw"}
interprovincial_clm = 0 if interprovincial_clm == False else 1
interprovincial_clm

0

In [ ]:
#@title Número de asientos

asientos_clm = 2 #@param {type:"slider", min:0, max:50, step:1}

In [ ]:
#@title Número de viajes que harías

viajes_clm = 1 #@param {type:"slider", min:0, max:25, step:1}

In [ ]:
dic_caracteristicas_clm = {'DIA':dia_clm,'ORIGEN':origen_clm,'VIAJES_OFERTADOS':viajes_clm,'ASIENTOS_OFERTADOS':asientos_clm, 'MES':mes_clm,'AÑO':año_clm,'SEMANA':semana_clm, 'INTERPROVINCIAL':interprovincial_clm}
df_viaje_clm = pd.DataFrame(dic_caracteristicas_clm, index=[0])
df_viaje_clm = df_viaje_clm.set_index('DIA')
df_viaje_clm

,ORIGEN,VIAJES_OFERTADOS,ASIENTOS_OFERTADOS,MES,AÑO,SEMANA,INTERPROVINCIAL
DIA,,,,,,,
2022-01-25,37,1,2,1,2022,4,0


In [ ]:
pred_clm =  clf_rf.predict(df_viaje_clm)
ranking = dict(Counter(pred_clm))
best = list(ranking.keys())[0]

destino_final_clm = destino_dic_clm[best]

print("Destino con mas probabilidad de que tenga viaje:", destino_final_clm)

Destino con mas probabilidad de que tenga viaje: Talavera de la Reina


## Predicción para el usuario en Andalucía

In [ ]:
from collections import Counter

In [ ]:
orig_andalucia = andalucia.ORIGEN.astype('category')
origen_dic_andalucia = dict(enumerate(orig_andalucia.cat.categories))

desti_andalucia = andalucia.DESTINO.astype('category')
destino_dic_andalucia = dict(enumerate(desti_andalucia.cat.categories))

interprovincial_andalucia = andalucia.INTERPROVINCIAL.astype('category')
interprovincial_dic_andalucia =dict(enumerate(interprovincial_andalucia.cat.categories))

In [ ]:
#@title ORIGEN

origen_andalucia = 'Lepe' #@param {type:"string"}

origen_andalucia = list(origen_dic_andalucia.keys())[list(origen_dic_andalucia.values()).index(origen_andalucia)]
origen_andalucia

276

In [ ]:
from datetime import datetime

#@title FECHA DE VIAJE
dia_andalucia = '2022-01-26' #@param {type:"date"}


date_andalucia = pd.Timestamp(dia_andalucia)
mes_andalucia = date_andalucia.month

semana_andalucia = date_andalucia.week
año_andalucia = date_andalucia.year
print(mes_andalucia,año_andalucia,semana_andalucia)

1 2022 4


In [ ]:
#@title Interprovincial

interprovincial = False #@param ["False", "True"] {type:"raw"}
interprovincial = 0 if interprovincial == False else 1
interprovincial

0

In [ ]:
#@title Número de asientos

asientos = 2 #@param {type:"slider", min:0, max:50, step:1}

In [ ]:
#@title Número de viajes que harías

viajes = 1 #@param {type:"slider", min:0, max:25, step:1}

In [ ]:
dic_caracteristicas_andalucia = {'DIA':dia_andalucia,'ORIGEN':origen_andalucia,'VIAJES_OFERTADOS':viajes,'ASIENTOS_OFERTADOS':asientos, 'MES':mes_andalucia,'AÑO':año_andalucia,'SEMANA':semana_andalucia, 'INTERPROVINCIAL':interprovincial}
df_viaje_andalucia = pd.DataFrame(dic_caracteristicas_andalucia, index=[0])
df_viaje_andalucia = df_viaje_andalucia.set_index('DIA')
df_viaje_andalucia

,ORIGEN,VIAJES_OFERTADOS,ASIENTOS_OFERTADOS,MES,AÑO,SEMANA,INTERPROVINCIAL
DIA,,,,,,,
2022-01-26,276,1,2,1,2022,4,0


In [ ]:
pred_andalucia =  clf.predict(df_viaje_andalucia)
ranking = dict(Counter(pred_andalucia))
best = list(ranking.keys())[0]

destino_final_andalucia = destino_dic_andalucia[best]

print("Destino con mas probabilidad de que tenga viaje:", destino_final_andalucia)

Destino con mas probabilidad de que tenga viaje: Sevilla
